In [ ]:
# Project by Jordan Ahern, Ryan Edmonds and Carson Sasso

In [2]:
from IPython.core.interactiveshell import InteractiveShell
from ipywidgets import widgets 
import json
import urllib.parse
import urllib.request  
from geopy.geocoders import Nominatim
# User has to install geopy

# pretty print all cell's output and not just the last ones
InteractiveShell.ast_node_interactivity = "all"

In [3]:
# Basic HTML below adds instructions to the top of our code. Includes elements such as bold face type. 
widgets.HTML(value = """<p><b>This code tells the user what the average electric bill would be for a household of similar size in their area</p></b><br>
<p><b>Instructions:&nbsp;</p></b> 
<ol>
    <li><mark>User must install Geopy</mark></li>
    <li>Select the number of people in the household</li>
    <li>Input your longitude and latitude <u>(note W and S should be entered as negative values)</u></li>
    <li>Click the RUN button, the result will appear in red</li>
</ol>
<p><b>Sample Locations:</b></p>
<ol>
    <li>Boston College: &nbsp; Latittude: 42.33 , &nbsp;Longitude: -71.16</li>
    <li>Phoenix Arizona: Latitude: 33.44 , &nbsp;Longitude: -112.07</li>
    <li>Miami Florida:&nbsp; &nbsp; Latitude: 25.76 ,&nbsp; Longitude: -80.19</li>
</ol>
<p><br></p>""")

input_text = widgets.Text(description = "Latitude:")   #creates textbox for latitude 
input_text2 = widgets.Text(description = "Longitude:") #creates textbox for longitude

average_usage = 100    #daily average kilowat hours per person according to the EIA
family_options = [1,2,3,4,5,6,7,8,9,10]  #establishes range of options for family size
choice_out = widgets.Output()  #establishes range of options for family size
choose_options = widgets.Select(
    options= [1,2,3,4,5,6,7,8,9,10],  #options for the family size select widget 
    value=None,
    description='Select family size' #description for select family size
) 
def colored(r, g, b, text): #Allows us to make our final print statement green so it stands out and looks pretty
    return "\033[38;2;{};{};{}m{} \033[38;2;255;255;255m".format(r, g, b, text)

def binder(arg): #Function for our latitude and Longitude function
    global latcord      #For our lat/long widget we make it a global variable in order to hold its value in whole code - easy troubleshooting
    global longcord     
    latcord = float(input_text.value) # change string input into a float
    longcord = float(input_text2.value)
    global price # define price inn whole code
    key = 'K4iqHWH5XeOYh60uaQPD8AnfxHe72FEW4hYwvvAU'  #key for our powergrid API 
    lat = f"&lat={latcord}"   #In the API, this was used to pull our latitude data
    long = f"&lon={longcord}" #In the API, this was used to pull our longitude data 
    site = "https://developer.nrel.gov/api/utility_rates/v3.json?api_key=" 
    response_object = urllib.request.urlopen(site+key+lat+long) # set responce object to propper format
    response_string = response_object.read()     #code to read in our data from the API 
    response_as_dict = json.loads(response_string) #converts our data to a dictionary
    price = response_as_dict['outputs']['residential'] #pulls the price of per kilowatt hour value out of the dictionary 
    
    



def estimated_price(c):  #function for determining the average price for certain areas
    with choice_out:        
    # initialize Nominatim API 
        global location #define variables as global for troubleshooting
        global average_cost
        geolocator = Nominatim(user_agent="geoapiExercises") #
        # Latitude & Longitude input
        try:
            Latitude = str(latcord) #geolocater requires lat long be strings
            Longitude = str(longcord)
            location = geolocator.reverse(Latitude+","+Longitude) #use geolocater to set location
            location = (str(location)).split(",") #make location a list
            location = location[4]+","+location[5] #pull only useful info from list
            family_size = choose_options.value #sets the family size equal to the value that is selected in choose options
            average_cost = ((average_usage*family_size+250)*price) #Calculates cost, factoring in a base value / intercept of 80KwH per household
            print(colored(0,100,0,f'The average monthly electric bill for a family of {family_size} in {location} is ${int(average_cost)}.00'))
        except: #if there is no data for that location, print the below statement
            print(colored(200,100,0,f'No data for this location. Check the Coordinates. If problem persists try a nearby location for best estimate'))
        
my_button = widgets.Button(description = "RUN") #makes button widget
my_button.on_click(binder) #run the two functions when the button is clicked
my_button.on_click(estimated_price)
choice_out #display select widget


def make_boxes(): #used this function to organize our widgets visually
    vbox1 = widgets.VBox([choose_options,  my_button]) #helps establish our choose_options widget in the far left of the screen
    vbox2 = widgets.VBox([input_text, input_text2])  #helps move our latitude and longitude functions over to the right of the screen
    return vbox1, vbox2
vbox1, vbox2 = make_boxes()
widgets.HBox([vbox1, vbox2])#returns the two widget boxes 


HTML(value='<p><b>This code tells the user what the average electric bill would be for a household of similar …

Output()